In [261]:
import pandas as pd
import matplotlib.pyplot as pl
from gensim.models import Word2Vec,KeyedVectors
import nltk
from nltk.corpus import stopwords
import re

from sklearn.feature_extraction.text import CountVectorizer,TfidfVectorizer
from sklearn.preprocessing import LabelEncoder
from sklearn.svm import SVC
from sklearn.linear_model import PassiveAggressiveClassifier
from sklearn.naive_bayes import MultinomialNB
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import SGDClassifier

from sklearn.metrics.pairwise import cosine_similarity,cosine_distances

import numpy as np

In [262]:
df=pd.read_excel('data.xlsx')

In [263]:
df.head()

,Category,Tag,Question,Answer
0,My Account,Rating and Reviews,How to Review Product as well as Seller?,Everything is right here. Please check this ou...
1,My Account,Rating and Reviews,How to Filter Search Results with Product Rati...,Products can also be filtered according to the...
2,My Account,Rating and Reviews,What are Ratings & Reviews and how do I write ...,Daraz customers have the option of rating and ...
3,My Account,Product Warranty,How do I know if a product comes with free ins...,Unfortunately we are not providing the Free In...
4,My Account,Product Warranty,How do I know if a product comes with warranty?,If a warranty is offered on a product the warr...


In [264]:
txt=df['Answer'][2]
txt

'Daraz customers have the option of rating and reviewing their product and seller experiences.,The customers can both leave star ratings and add text to describe what they liked or disliked about their experience.'

In [265]:
txt.split(',')

['Daraz customers have the option of rating and reviewing their product and seller experiences.',
 'The customers can both leave star ratings and add text to describe what they liked or disliked about their experience.']

In [266]:
df['Question'].head()

0             How to Review Product as well as Seller?
1    How to Filter Search Results with Product Rati...
2    What are Ratings & Reviews and how do I write ...
3    How do I know if a product comes with free ins...
4      How do I know if a product comes with warranty?
Name: Question, dtype: object

In [267]:
def cleantext(txt):
    txt=txt.split()
    txt=[i.lower() for i in txt if i not in stopwords.words('english') ]
    txt=' '.join(txt)
    txt=re.sub(r'[^A-Za-z0-9]',' ',txt)
    txt=' '.join(txt.split())
    return txt 
    
    
cleantext('My name  is  rishav..') 
df['Cleaned']=df['Question'].apply(lambda x :cleantext(x))

In [268]:
df['Cleaned'].head(10)

0                how review product well seller
1     how filter search results product ratings
2     what ratings reviews i write good reviews
3    how i know product comes free installation
4             how i know product comes warranty
5     i lost warranty card how i claim warranty
6        are hidden costs charges i order daraz
7                   are prices daraz negotiable
8            why i see different prices product
9                            how contact vendor
Name: Cleaned, dtype: object

In [269]:
vectorizer=TfidfVectorizer()
X_=vectorizer.fit_transform(df['Cleaned']).toarray()

cv=CountVectorizer()
X_cv_=cv.fit_transform(df['Cleaned']).toarray()

In [270]:
X_[:2]

array([[0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.28192401, 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.  

In [271]:
X_.shape

(44, 123)

In [272]:
labelencoder=LabelEncoder()
df['Label']=labelencoder.fit_transform(df['Category'])

In [273]:
df.head()

,Category,Tag,Question,Answer,Cleaned,Label
0,My Account,Rating and Reviews,How to Review Product as well as Seller?,Everything is right here. Please check this ou...,how review product well seller,1
1,My Account,Rating and Reviews,How to Filter Search Results with Product Rati...,Products can also be filtered according to the...,how filter search results product ratings,1
2,My Account,Rating and Reviews,What are Ratings & Reviews and how do I write ...,Daraz customers have the option of rating and ...,what ratings reviews i write good reviews,1
3,My Account,Product Warranty,How do I know if a product comes with free ins...,Unfortunately we are not providing the Free In...,how i know product comes free installation,1
4,My Account,Product Warranty,How do I know if a product comes with warranty?,If a warranty is offered on a product the warr...,how i know product comes warranty,1


In [274]:
y=df['Label']

In [275]:
from sklearn.linear_model import LogisticRegression
logistic=LogisticRegression().fit(X_,y)
passive=PassiveAggressiveClassifier().fit(X_,y)
svm=SVC().fit(X_,y)
naive=MultinomialNB().fit(X_,y)
dectr=DecisionTreeClassifier().fit(X_,y)
rand_for=RandomForestClassifier().fit(X_,y)
sgd=SGDClassifier().fit(X_,y)

In [276]:
print(logistic.score(X_,y))
print(passive.score(X_,y))
print(svm.score(X_,y))
print(naive.score(X_,y))
print(dectr.score(X_,y))
print(rand_for.score(X_,y))
print(sgd.score(X_,y))

0.9090909090909091
1.0
1.0
0.9318181818181818
1.0
1.0
1.0


In [277]:
def return_tfidf_data(text):
    return vectorizer.transform([text])

def return_cv_data(text):
    return cv.transform([text])

def return_vectorized_data(text):
    text=cleantext(text)
    print('Asked Question: ',text)
    print('-----------------------------')
    text=return_tfidf_data(text)
    return text.toarray()

def return_countvectorized_data(text):
    text=cleantext(text)
    print('Asked Question: ',text)
    print('-----------------------------')
    text=return_cv_data(text)
    return text.toarray()
    
def predict_label(model):
    prediction=model.predict(temp)[0]
    return prediction
    
def find_relevant_ques_ans_model(temp,items):
    for i in models:
        prediction=predict_label(i)
        index=df[df['Label']==prediction].index

        cos_sim=[cosine_similarity(j.reshape(1,-1),temp.reshape(1,-1)) for j in X_[df[df['Label']==prediction].index]]
        cos_sim=np.array(cos_sim)

        cos_sim=cos_sim.flatten()
        temp_df=df.iloc[index,:].copy()
        temp_df['cosine_sim']=cos_sim
        temp_df=temp_df.sort_values('cosine_sim',ascending=False)
        print(i)
        print(temp_df[['Question','cosine_sim']].head(items))
        print('----------------------------------------------------------')
    
temp=return_vectorized_data('How to check warrenty of product?')
find_relevant_ques_ans_model(temp,3)

Asked Question:  how check warrenty product
-----------------------------
LogisticRegression()
                                            Question  cosine_sim
4    How do I know if a product comes with warranty?    0.313713
0           How to Review Product as well as Seller?    0.289147
3  How do I know if a product comes with free ins...    0.267625
----------------------------------------------------------
PassiveAggressiveClassifier()
                                            Question  cosine_sim
4    How do I know if a product comes with warranty?    0.313713
0           How to Review Product as well as Seller?    0.289147
3  How do I know if a product comes with free ins...    0.267625
----------------------------------------------------------
SVC()
                                             Question  cosine_sim
32  Problems with Check Out/ or any issues encount...    0.339369
27                          How do I cancel my order?    0.163451
43         How can I find oversea

In [278]:
temp=return_vectorized_data('How to check product installation?')
find_relevant_ques_ans(temp,3)

Asked Question:  how check product installation
-----------------------------
LogisticRegression()
                                            Question  cosine_sim
3  How do I know if a product comes with free ins...    0.502639
4    How do I know if a product comes with warranty?    0.249904
0           How to Review Product as well as Seller?    0.230336
----------------------------------------------------------
PassiveAggressiveClassifier()
                                            Question  cosine_sim
3  How do I know if a product comes with free ins...    0.502639
4    How do I know if a product comes with warranty?    0.249904
0           How to Review Product as well as Seller?    0.230336
----------------------------------------------------------
SVC()
                                            Question  cosine_sim
3  How do I know if a product comes with free ins...    0.502639
4    How do I know if a product comes with warranty?    0.249904
0           How to Review Produc

In [279]:
# cos_sim=cosine_similarity(A.reshape(1,-1),B.reshape(1,-1))
# print (f"Cosine Similarity between A and B:{cos_sim}")
# print (f"Cosine Distance between A and B:{1-cos_sim}")

In [280]:
cos_sim=[cosine_similarity(X_[i].reshape(1,-1),temp.reshape(1,-1)) for i in range(len(X_))]
cos_sim=np.array(cos_sim)

cos_sim=cos_sim.flatten()

In [281]:
df_temp=df.copy()
df_temp['Cos_sim']=cos_sim

In [282]:
df_temp.head()

,Category,Tag,Question,Answer,Cleaned,Label,Cos_sim
0,My Account,Rating and Reviews,How to Review Product as well as Seller?,Everything is right here. Please check this ou...,how review product well seller,1,0.230336
1,My Account,Rating and Reviews,How to Filter Search Results with Product Rati...,Products can also be filtered according to the...,how filter search results product ratings,1,0.208753
2,My Account,Rating and Reviews,What are Ratings & Reviews and how do I write ...,Daraz customers have the option of rating and ...,what ratings reviews i write good reviews,1,0.000000
3,My Account,Product Warranty,How do I know if a product comes with free ins...,Unfortunately we are not providing the Free In...,how i know product comes free installation,1,0.502639
4,My Account,Product Warranty,How do I know if a product comes with warranty?,If a warranty is offered on a product the warr...,how i know product comes warranty,1,0.249904


In [283]:
df_temp.sort_values('Cos_sim',ascending=False).head()

,Category,Tag,Question,Answer,Cleaned,Label,Cos_sim
3,My Account,Product Warranty,How do I know if a product comes with free ins...,Unfortunately we are not providing the Free In...,how i know product comes free installation,1,0.502639
32,Ordering,Big mart,Problems with Check Out/ or any issues encount...,If you are facing any error while placing the ...,problems check out issues encountered,2,0.270342
15,DarazMall,What is Darazmall,How can I identify a DarazMall product in the ...,"You can identify"" DarazMall"" products by the ""...",how i identify darazmall product website,0,0.250737
4,My Account,Product Warranty,How do I know if a product comes with warranty?,If a warranty is offered on a product the warr...,how i know product comes warranty,1,0.249904
0,My Account,Rating and Reviews,How to Review Product as well as Seller?,Everything is right here. Please check this ou...,how review product well seller,1,0.230336


In [336]:
def return_response(temp_df):
    most_relevant=temp_df.copy()
    relevant_index=most_relevant.index[0]
    response_list=df.iloc[relevant_index,:]['Answer']
    
    try:
        responses=df.iloc[relevant_index,:]['Answer'].split(',')
        print('Response: ',np.random.choice(responses))
    except:
        print('Response: ',responses)
    
#For TFIDF
def find_relevant(temp,items):
    prediction=predict_label(svm)
    index=df[df['Label']==prediction].index

    cos_sim=[cosine_similarity(j.reshape(1,-1),temp.reshape(1,-1)) for j in X_[index]]
    cos_sim=np.array(cos_sim)

    cos_sim=cos_sim.flatten()
    temp_df=df.iloc[index,:].copy()
    temp_df['cosine_sim']=cos_sim
    temp_df=temp_df.sort_values('cosine_sim',ascending=False)
    return(temp_df[['Question','cosine_sim']].head(items)),return_response(temp_df)

#For CV
def find_relevant_cv(temp,items):
    prediction=predict_label(svm)
    index=df[df['Label']==prediction].index
    cos_sim=[cosine_similarity(j.reshape(1,-1),temp.reshape(1,-1)) for j in X_cv_[index]]
    cos_sim=np.array(cos_sim)

    cos_sim=cos_sim.flatten()
    temp_df=df.iloc[index,:].copy()
    temp_df['cosine_sim']=cos_sim
    temp_df=temp_df.sort_values('cosine_sim',ascending=False)
    return(temp_df[['Question','cosine_sim']].head(items)),return_response(temp_df)


In [337]:
#Using Countvectorizer
temp=return_countvectorized_data('How to know good reviews?')
print(temp)
returned_df,relevant_answer=find_relevant_cv(temp,3)
returned_df

Asked Question:  how know good reviews
-----------------------------
[[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
  0 0 0 0 0 0 0 0 1 0 0 0 1 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0
  0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
  0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]]
Response:  If your order is not yet on its way to you and is eligible for cancellation  "Cancel Button" will indicate on your desired order that you wish to cancel.


,Question,cosine_sim
27,How do I cancel my order?,0.288675
43,How can I find overseas products on Daraz?,0.223607
40,How do I remove an item from my shopping cart?,0.223607


In [338]:
temp=return_countvectorized_data('How to write my name?')
returned_df,relevant_answer=find_relevant_cv(temp,3)
returned_df

Asked Question:  how write name
-----------------------------
Response:  If your order is not yet on its way to you and is eligible for cancellation  "Cancel Button" will indicate on your desired order that you wish to cancel.


,Question,cosine_sim
27,How do I cancel my order?,0.408248
43,How can I find overseas products on Daraz?,0.316228
40,How do I remove an item from my shopping cart?,0.316228


In [339]:
#Using TFIDF Vectorizer
temp=return_vectorized_data('How to check product installation?')
returned_df,relevant_answer=find_relevant(temp,3)
returned_df

Asked Question:  how check product installation
-----------------------------
Response:  Unfortunately we are not providing the Free Installation Services for now


,Question,cosine_sim
3,How do I know if a product comes with free ins...,0.502639
4,How do I know if a product comes with warranty?,0.249904
0,How to Review Product as well as Seller?,0.230336


In [340]:
temp=return_vectorized_data('why i see different prices product?')
returned_df,relevant_answer=find_relevant(temp,3)
returned_df

Asked Question:  see different prices product
-----------------------------
Response:  Daraz is a marketplace. We have a huge seller base and each one sources their product differently due to which prices vary for the same product but you can choose depending on your preference as the product quality remains the same.


,Question,cosine_sim
8,Why do I see different prices for the same pro...,0.922422
7,Are the prices on Daraz negotiable?,0.279414
4,How do I know if a product comes with warranty?,0.140928


In [328]:
returned_df['Question'][32]

'Problems with Check Out/ or any issues encountered?'

In [341]:
temp=return_vectorized_data('can i play daraz first games desktop too?')
returned_df,relevant_answer=find_relevant(temp,3)
returned_df

Asked Question:  play daraz first games desktop too
-----------------------------
Response:  No you can only play Daraz First Games on Daraz Android App.


,Question,cosine_sim
21,Can I play Daraz First Games on Desktop too?,0.944237
22,What is Daraz First Games?,0.589203
18,Why am I not able to see my Collectible Vouche...,0.000000


In [343]:
temp=return_countvectorized_data('can i play daraz first games desktop too?')
returned_df,relevant_answer=find_relevant_cv(temp,3)
returned_df

Asked Question:  play daraz first games desktop too
-----------------------------
Response:   We are implementing WHO prescribed precautionary measures for the situation.



,Question,cosine_sim
29,Are the employees at Daraz safe?,0.204124
43,How can I find overseas products on Daraz?,0.182574
39,Do I need an account to shop on Daraz?,0.182574
